In [1]:
# cd("/Users/elfflorin/Documents/Projects/julia.hw/jpie-v0.4"); include("setup.jl"); using brml

###condp() functionality

In [2]:
# generate a toy matrix to follow its cells movements
N = 4; M = 2; L = 3
p = zeros(N, M, L)
for z = 1:L
    for y = 1:M
        for x = 1:N
            p[x, y, z] = x + (y - 1) * N  + (z - 1) * 10^(z - 1)
        end
    end
end
p

4x2x3 Array{Float64,3}:
[:, :, 1] =
 1.0  5.0
 2.0  6.0
 3.0  7.0
 4.0  8.0

[:, :, 2] =
 11.0  15.0
 12.0  16.0
 13.0  17.0
 14.0  18.0

[:, :, 3] =
 201.0  205.0
 202.0  206.0
 203.0  207.0
 204.0  208.0

In [3]:
@time permutedims(p, (3, 1, 2))

3x4x2 Array{Float64,3}:
[:, :, 1] =
   1.0    2.0    3.0    4.0
  11.0   12.0   13.0   14.0
 201.0  202.0  203.0  204.0

[:, :, 2] =
   5.0    6.0    7.0    8.0
  15.0   16.0   17.0   18.0
 205.0  206.0  207.0  208.0

In [4]:
@time reshape(p, (3, 4, 2))

3x4x2 Array{Float64,3}:
[:, :, 1] =
 1.0  4.0   7.0  12.0
 2.0  5.0   8.0  13.0
 3.0  6.0  11.0  14.0

[:, :, 2] =
 15.0   18.0  203.0  206.0
 16.0  201.0  204.0  207.0
 17.0  202.0  205.0  208.0

In [5]:
# COMMA makes a difference
println(typeof([3, 1]), " - size: " , size([3, 1]), " - 2x1 column vector")
[3, 1]

0.117024 seconds (55.74 k allocations: 2.873 MB)
  0.033074 seconds (20.04 k allocations: 1.005 MB)
Array{

2-element Array{Int64,1}:
 3
 1

In [6]:
println(typeof([3 1]), " - size: " , size([3 1]),  " - 1x2 row vector")
[3 1]

Int64,1} - size: (2,) - 2x1 column vector
Array{

1x2 Array{Int64,2}:
 3  1

In [7]:
# indices for dimensions to condition on
dindices = [3 1]

Int64,2} - size: (1,2) - 1x2 row vector


1x2 Array{Int64,2}:
 3  1

In [8]:
# how many dimensions are in p? allvars
# size() returns a vector with length of each dimension
allvars = 1: length(size(p))
sizevars = size(p)
typeof(sizevars)

Tuple{Int64,Int64,Int64}

In [9]:
# what are the dimensions not included in conditioning
condvars = setdiff(allvars, dindices)

1-element Array{Int64,1}:
 2

####_transform the raw data into probability distributions using 2 sets: conditioning (dindices) and conditioned (condvars)_

In [10]:
# create new matrix by swapping the matrix elements based on 
# the permutation of the dimension indices 
# in this case from NxMxL to LxNxM
# grouping the elements in LxN sets along the M dimension
newp = deepcopy(permutedims(p, [dindices condvars]))

3x4x2 Array{Float64,3}:
[:, :, 1] =
   1.0    2.0    3.0    4.0
  11.0   12.0   13.0   14.0
 201.0  202.0  203.0  204.0

[:, :, 2] =
   5.0    6.0    7.0    8.0
  15.0   16.0   17.0   18.0
 205.0  206.0  207.0  208.0

In [11]:
# reshape the new matrix for calculating the elements sum
# for the condvars (M = 2 in this case) sets of elements 
# in column vector format with condvars columns
# use prod() to use the cardinality for each set as dimension
# in this case (L*N) for each of the condvars (M) columns
# each of the condvars (M) sets is normalized as a distribution
newp = reshape(newp, prod(sizevars[dindices]), prod(sizevars[condvars]))

12x2 Array{Float64,2}:
   1.0    5.0
  11.0   15.0
 201.0  205.0
   2.0    6.0
  12.0   16.0
 202.0  206.0
   3.0    7.0
  13.0   17.0
 203.0  207.0
   4.0    8.0
  14.0   18.0
 204.0  208.0

In [12]:
# calculate the sum of all elements in the 
# condvars (M=2) sets; replicate
# the results in a matrix similar to current data matrix
sump = repmat(sum(newp, 1), size(newp, 1), 1)

12x2 Array{Float64,2}:
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0
 870.0  918.0

In [13]:
int(sum(newp[:,2] ./ sump[:, 2]))

1

In [14]:
# normalize the condvars (M) sets in the current data matrix
probp = newp ./ sump

12x2 Array{Float64,2}:
 0.00114943  0.00544662
 0.0126437   0.0163399 
 0.231034    0.223312  
 0.00229885  0.00653595
 0.0137931   0.0174292 
 0.232184    0.224401  
 0.00344828  0.00762527
 0.0149425   0.0185185 
 0.233333    0.22549   
 0.0045977   0.0087146 
 0.016092    0.0196078 
 0.234483    0.22658   

 in depwarn at deprecated.jl:73
 in int at deprecated.jl:50
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/elfflorin/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /Users/elfflorin/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[13], in expression starting on line 1


In [15]:
# check each set is a probability distribution
sum(probp, 1)

1x2 Array{Float64,2}:
 1.0  1.0

####_rollback the operations to bring the data back into its original matrix format NxMxL_

In [16]:
# reshape to previous reshaped format 
# in this case from (L*N)x(1*M) to LxNxM
probp = reshape(probp, sizevars[[dindices condvars]])

3x4x2 Array{Float64,3}:
[:, :, 1] =
 0.00114943  0.00229885  0.00344828  0.0045977
 0.0126437   0.0137931   0.0149425   0.016092 
 0.231034    0.232184    0.233333    0.234483 

[:, :, 2] =
 0.00544662  0.00653595  0.00762527  0.0087146
 0.0163399   0.0174292   0.0185185   0.0196078
 0.223312    0.224401    0.22549     0.22658  

In [17]:
probp = ipermutedims(probp, [dindices condvars])

4x2x3 Array{Float64,3}:
[:, :, 1] =
 0.00114943  0.00544662
 0.00229885  0.00653595
 0.00344828  0.00762527
 0.0045977   0.0087146 

[:, :, 2] =
 0.0126437  0.0163399
 0.0137931  0.0174292
 0.0149425  0.0185185
 0.016092   0.0196078

[:, :, 3] =
 0.231034  0.223312
 0.232184  0.224401
 0.233333  0.22549 
 0.234483  0.22658 

In [18]:
testp = sum(probp, 3)

4x2x1 Array{Float64,3}:
[:, :, 1] =
 0.244828  0.245098
 0.248276  0.248366
 0.251724  0.251634
 0.255172  0.254902

In [19]:
# final check
sum(testp, 1)

1x2x1 Array{Float64,3}:
[:, :, 1] =
 1.0  1.0

### iu-hu-huu!!!